Evaluating how much value a player gives relative to their salary 

Cleaning and Merging 2 Sheets (NBA_Cleaned and NBA_Contracts)

In [5]:
import pandas as pd

# Load CSV files
stat_df = pd.read_csv("NBA_Cleaned.csv")
contract_df = pd.read_csv("NBA_Contracts.csv")

# 1: Clean Salary Column by removing commas and converting to floats
contract_df['Salary (USD)'] = contract_df["Salary (USD)"].replace(",", "", regex=True).astype(float)

# 2: Rename columns so 'Player' is consistent 
contract_df.rename(columns={"Name": "Player"}, inplace=True)

# 3: Merge both datasets on Player and Season
merged_df = pd.merge(stat_df, contract_df[['Player', 'Salary (USD)', 'Season']], on=['Player', 'Season'], how='left')
# Step 4: Create a column for salary in millions (if not done already)
merged_df['Salary (Millions)'] = merged_df['Salary (USD)'] / 1_000_000

# 5: Save to new CSV or preview
merged_df.to_csv("Merged_NBA_Data.csv", index=False)
print(merged_df.head())


                  Player  Age Team Pos   G  GS    MP    FG   FGA    FG%  ...  \
0          Stephen Curry   32  GSW  PG  63  63  34.2  10.4  21.7  0.482  ...   
1           Bradley Beal   27  WAS  SG  60  60  35.8  11.2  23.0  0.485  ...   
2         Damian Lillard   30  POR  PG  67  67  35.8   9.0  19.9  0.451  ...   
3            Joel Embiid   26  PHI   C  51  51  31.1   9.0  17.6  0.513  ...   
4  Giannis Antetokounmpo   26  MIL  PF  61  61  33.0  10.3  18.0  0.569  ...   

   AST  STL  BLK  TOV   PF   PTS                     Awards  Season  \
0  5.8  1.2  0.1  3.4  1.9  32.0              MVP-3,AS,NBA1    2021   
1  4.4  1.2  0.4  3.1  2.3  31.3                    AS,NBA3    2021   
2  7.5  0.9  0.3  3.0  1.5  28.8              MVP-7,AS,NBA2    2021   
3  2.8  1.0  1.4  3.1  2.4  28.5  MVP-2,DPOY-7,AS,NBA2,DEF2    2021   
4  5.9  1.2  1.2  3.4  2.8  28.1  MVP-4,DPOY-5,AS,NBA1,DEF1    2021   

   Salary (USD)  Salary (Millions)  
0    43006362.0          43.006362  
1    28751775.0   

Creating a Fantasy Value Score

In [6]:
def fantasy_score(row):
    # does not allow any players with 0 games played
    if row['G'] == 0 or pd.isna(row['G']):
        return 0 
    
    #Normalize for Volume (players with less GP's may look better)
    pts = row['PTS'] / row['G']
    ast = row['AST'] / row['G']
    reb = row['TRB'] / row['G']
    stl = row['STL'] / row['G']
    blk = row['BLK'] / row['G']
    tov = row['TOV'] / row['G']
    fg = row["FG"] if not pd.isna(row["FG"]) else 0
    three = fg = row["3P%"] if not pd.isna(row["3P%"]) else 0
    pos = row['Pos']

    # Position-specific weights
    # PGs are judged on playmaking and 3P% .Cs are judged more on interior dominance and FG%. SF and PF are balanced

    if 'PG' in pos:
        score = pts + 2 * ast + 1 * reb + 2 * stl + 0.5 * blk - tov + 2 * fg + 2.5 * three
    elif 'SG' in pos:
        score = 1.1 * pts + 1.5 * ast + 0.8 * reb + 2 * stl + 0.5 * blk - tov + 1.5 * fg + 3 * three
    elif 'SF' in pos:
        score = 1 * pts + 1.2 * ast + 1.2 * reb + 1.5 * stl + 1 * blk - tov + 1.5 * fg + 2 * three
    elif 'PF' in pos:
        score = 1 * pts + 1 * ast + 1.4 * reb + 1 * stl + 1.5 * blk - tov + 2 * fg + 1 * three
    elif 'C' in pos:
        score = 0.9 * pts + 0.6 * ast + 2 * reb + 0.5 * stl + 2.5 * blk - tov + 3 * fg + 0.5 * three
    else:
        score = pts + ast + reb + stl + blk - tov + fg + three  # fallback

    return score

# Step 6: Apply the fantasy score function
merged_df['FantasyScore'] = merged_df.apply(fantasy_score, axis=1)

# Step 7: Calculate value per $1 million
merged_df['ValuePerMillion'] = merged_df['FantasyScore'] / merged_df['Salary (Millions)']

# (Optional) Save or view
merged_df.to_csv("Final_NBA_Valued.csv", index=False)
print(merged_df[['Player', 'Pos', 'FantasyScore', 'Salary (Millions)', 'ValuePerMillion']].head())

                  Player Pos  FantasyScore  Salary (Millions)  ValuePerMillion
0          Stephen Curry  PG      2.658786          43.006362         0.061823
1           Bradley Beal  SG      2.308667          28.751775         0.080296
2         Damian Lillard  PG      2.460246          31.626953         0.077790
3            Joel Embiid   C      2.288716          29.542010         0.077473
4  Giannis Antetokounmpo  PF      1.712279          27.528090         0.062201
